# DB Populate Notebook

This notebook populates the database with a variety of apps, records, and
feedback results. It is used primarily for database migration testing.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

# Enables: Debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

In [ ]:
# ! pip install llama_index==0.9.15.post2
# ! pip install pydantic==2.5.2 pydantic_core==2.14.5

In [ ]:
# To test out DB migrations, copy one of the older db dumps to this folder first:

! ls ../../release_dbs/
! cp ../../release_dbs/0.1.2/default.sqlite default.sqlite
# ! rm default.sqlite

In [ ]:
from concurrent.futures import as_completed
import json
import os
from pathlib import Path
from time import sleep

import dotenv
from tqdm.auto import tqdm

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.endpoint.base import Endpoint
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.schema import Cost
from trulens_eval.schema import FeedbackMode
from trulens_eval.schema import Record
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

In [ ]:
# Setup Tru and/or dashboard.

tru = Tru(database_redact_keys=True)

# tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

Tru().migrate_database()

from trulens_eval.database.migrations.db_data_migration import _sql_alchemy_serialization_asserts
_sql_alchemy_serialization_asserts(tru.db)

# Feedbacks

In [ ]:
# Dummy endpoint

dummy = Dummy(
    loading_prob=0.1,
    freeze_prob=0.0, # we expect requests to have their own timeouts so freeze should never happen
    error_prob=0.01,
    overloaded_prob=0.1,
    rpm=6000
)

f_lang_match_dummy = Feedback(
    dummy.language_match
).on_input_output()

In [ ]:
# Huggingface endpoint
from trulens_eval import Huggingface

hugs = Huggingface()

f_lang_match_hugs = Feedback(hugs.language_match).on_input_output()

In [ ]:
# import inspect
# inspect.signature(Huggingface).bind()

In [ ]:
# Openai endpoint
from trulens_eval import OpenAI
openai = OpenAI()

f_relevance_openai = Feedback(openai.relevance).on_input_output()

In [ ]:
# Bedrock endpoint
# Cohere as endpoint
# LangChain as endpoint
# Litellm as endpoint

In [ ]:
feedbacks = [f_lang_match_hugs, f_lang_match_dummy, f_relevance_openai]

# _LangChain_ app

In [ ]:
from langchain_community.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI(temperature=0.9, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=llm,
)

# Conversational app puts it all together.
app_langchain = ConversationChain(
    llm=llm,
    memory=memory
)

from langchain.prompts import PromptTemplate
from trulens_eval.instruments import instrument
instrument.method(PromptTemplate, "format")

truchain = tru.Chain(app_langchain, app_id="langchain_app", feedbacks=feedbacks)

In [ ]:
with truchain as recs:
    print(app_langchain("Hello?"))

# Llama-index app

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(
    html_to_text=True
).load_data(["http://paulgraham.com/worked.html"])
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

trullama = tru.Llama(query_engine, app_id="llama_index_app")

In [ ]:
with trullama as recs:
    print(query_engine.query("Who is the author?"))

# Basic app

In [ ]:
from trulens_eval.tru_custom_app import instrument

def custom_application(prompt: str) -> str:
    return f"a useful response to {prompt}"

trubasic = tru.Basic(custom_application, app_id="basic_app", feedbacks=feedbacks)

In [ ]:
with trubasic as recs:
    print(trubasic.app("hello?"))

# Custom app

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp # our custom app

# Create custom app:
app_custom = CustomApp()

# Create trulens wrapper:
trucustom = tru.Custom(
    app=app_custom,
    app_id="custom_app",
    
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app_custom.respond_to_query,
    feedbacks=feedbacks
)


In [ ]:
with trucustom as recs:
    print(app_custom.respond_to_query("hello there"))